## Literature Review

What:
TextRank uses an extractive approach and is an unsupervised graph-based text summarization technique. 

How: 
The basic idea implemented by a graph-based ranking model is that of voting or recommendation.
When one vertex links to another one, it is basically casting a vote for that vertex. The higher the number of votes cast for a vertex, the higher the importance of that vertex.



## Implementation

### Prerequisite

untuk menjalankan project TextRank ini, diperlukan untuk :
1. men-download library stopwords Bahasa Indonesia: Python Sastrawi.
    Cara download library:
    1. Open terminal
    2. write on command line : "pip install Sastrawi"
2. men-download Python ROUGE
    Cara download library:
    1. Open terminal
    2. write on command line : "pip install py-rouge"

## Libraries

In [2]:
import nltk
import math

import urllib
import os, os.path
#import modular regular expression
import re
import string
# import StopWordRemoverFactory class
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from nltk.cluster.util import cosine_distance
import numpy as np

from operator import itemgetter 

### Open txt file and Preprocessing

Preprocessing methods used:
1. decode to UTF-8 -> standardisation
2. change all to lower case -> consistency purposes especially during stopwords removal process
3. remove all numbers -> avoid redundancy
4. remove all punctuation -> avoid redundancy
5. stopwords (using Python Sastrawi) -> avoid redundancy
6. Stemmer (using Python Sastrawi) -> easy filter

In [3]:
url = "https://raw.githubusercontent.com/jordhy97/final_project/master/data/reviews/reviews.txt"
file = urllib.request.urlopen(url)

#defining function getLines() = to get only the first 50 lines of review
def getLines(file):
    res = []
    i = 0

    while i < 50:
        res.append(str(file.readline(), "utf-8"))
        i+=1

    return res

file_lines = getLines(file)

stopword = StopWordRemoverFactory().create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

res_preprocessed_line = []

for line in file_lines:
#     preprocessed_line = line.decode("utf-8")
    preprocessed_line = line.lower()
    preprocessed_line = re.sub(r"\d+", "", preprocessed_line)
    preprocessed_line = preprocessed_line.translate(str.maketrans("","",string.punctuation))
    preprocessed_line = stopword.remove(preprocessed_line)
    preprocessed_line = stemmer.stem(preprocessed_line)
    print(preprocessed_line)
    res_preprocessed_line.append(preprocessed_line)

kotor debu tdk henti bersin awal masuk kamar kesan kamar kumuh wifi rusak lgsg check out
oke cuma air wastafel warna keruh mohon baik
kamar semut kamar mandi masalah buang air mampetsehingga air luber luar kamar
kamar mandi bau air bau
kamar bersih nyaman perlu perhati dan trus kroscheck trutama slot card pintu kamar batrai nya habis apa msih full kasi ada tamu lelah habis jalan jauh tunggu luar karena slotcard pintu kamar di perbaikin ulang tks smoga manfaat
tak sesuai espektasi kamar sempit pintu kamar mandi rusak cek in tunggu terlalu lama malam suara berisik dr atap kamar nyaman
booking traveloka room sedia kembali uang nya lamaaa
pesan kamar double bad dapat kamar twins bad
buruk kasur bekas sperma seprai jg air bau karat kasur per jg rusak sabun habis pasword wifi ga tau
baik puas masih kurang waktu boking airy rooms pilih bad tp saya cek in kasih bad mohon di tingkat terimah kasih airy rooma
kamar mandi kurang bersih kurang lengkap faslitasnya kamar mandi yang lumayan
lokasi sus

### Modelling

we first build a graph associated with the text, where the graph vertices are representative for the units to be ranked. The goal is to rank entire sentences, therefore, a vertex is added to the graph for each sentence in the text.

Here are some conditions a good similarity measure has to obey:

1. 0 <= similarity(A, B) <= 1
2. similarity(A, A) = 1
3. similarity(A, B) =/= 1 if A =/= B

Note: A and B are different sentences.

A widely used measure in Natural Language Processing is the Cosine Similarity. The Cosine Similarity computes the cosine of the angle between 2 vectors. If the vectors are identical, the cosine is 1.0. If the vectors are orthogonal, the cosine is 0.0. This means the cosine similarity is a measure we can use. NLTK implements cosine_distance, which is 1 - cosine_similarity. The concept of distance is opposite to similarity. Two identical vectors are located at 0 distance and are 100% similar.

#### Sentence Similarity using Cosine Similarity

In [4]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

#### Building the similarity Matrix

In [11]:
def build_similarity_matrix(res_preprocessed_line, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros((len(res_preprocessed_line), len(res_preprocessed_line)))
 
    for idx1 in range(len(res_preprocessed_line)):
        for idx2 in range(len(res_preprocessed_line)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(res_preprocessed_line[idx1], res_preprocessed_line[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S
 
print("Total number of reviews: ")
print(len(res_preprocessed_line))
print()

stop_words = ""
S = build_similarity_matrix(res_preprocessed_line, stop_words)    
print(S)

Total number of reviews: 
50

[[0.         0.02184822 0.02021487 ... 0.02207463 0.02021411 0.02016505]
 [0.02178269 0.         0.02098525 ... 0.02140803 0.01968865 0.02081242]
 [0.02021233 0.02104574 0.         ... 0.02177392 0.01723683 0.02091857]
 ...
 [0.02138501 0.02080162 0.02109634 ... 0.         0.01845196 0.02060326]
 [0.02208241 0.02157309 0.01883232 ... 0.02080743 0.         0.0198931 ]
 [0.02044759 0.02116751 0.02121433 ... 0.02156566 0.01846517 0.        ]]


#### Defining pagerank function

In [13]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P

#### Compute the Summary

In [18]:
sentence_ranks = pagerank(S)
 
print(sentence_ranks)
 
# Get the sentences ordered by rank
ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
print(ranked_sentence_indexes)
 
# Suppose we want the 5 most import sentences
SUMMARY_SIZE = 5
SELECTED_SENTENCES = sorted(ranked_sentence_indexes[:SUMMARY_SIZE])
print(SELECTED_SENTENCES)
 
# Fetch the most important sentences
summary = itemgetter(*SELECTED_SENTENCES)(res_preprocessed_line)
 
# Print the actual summary
for line in summary:
    print(''.join(line))

[0.02044427 0.02049617 0.02043572 0.01931903 0.02115599 0.02101363
 0.02007606 0.0199717  0.02018566 0.02037842 0.020355   0.02059009
 0.01827864 0.02031692 0.02095459 0.01957454 0.02031946 0.02071954
 0.02098467 0.02056576 0.02054052 0.0209514  0.0205534  0.01409787
 0.02110922 0.02047377 0.01975169 0.01956799 0.02046342 0.02089271
 0.02086726 0.0186872  0.02035053 0.02069653 0.01704985 0.02015594
 0.02036591 0.0211268  0.02000119 0.01966695 0.0166519  0.02058684
 0.02023277 0.02031293 0.0196461  0.01827864 0.02062807 0.02100153
 0.01896214 0.02019307]
[4, 37, 24, 5, 47, 18, 14, 21, 29, 30, 17, 33, 46, 11, 41, 19, 22, 20, 1, 25, 28, 0, 2, 9, 36, 10, 32, 16, 13, 43, 42, 49, 8, 35, 6, 38, 7, 26, 39, 44, 15, 27, 3, 48, 31, 12, 45, 34, 40, 23]
[4, 5, 24, 37, 47]
kamar bersih nyaman perlu perhati dan trus kroscheck trutama slot card pintu kamar batrai nya habis apa msih full kasi ada tamu lelah habis jalan jauh tunggu luar karena slotcard pintu kamar di perbaikin ulang tks smoga manfaat
ta

#### Take the top 5 summaries

In [38]:
def textrank(res, top_n=5, stopwords=None):
    """
    res_preprocessed_line = a list of airy review sentences 
    top_n = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    S = build_similarity_matrix(res_preprocessed_line, stop_words) 
    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(res_preprocessed_line)
    return summary

candidates = []

for idx, line in enumerate(textrank(res_preprocessed_line, stopwords=StopWordRemoverFactory().create_stop_word_remover())):
    print("%s. %s" % ((idx + 1), ''.join(line)))
    candidates.append(''.join(line))

1. kamar bersih nyaman perlu perhati dan trus kroscheck trutama slot card pintu kamar batrai nya habis apa msih full kasi ada tamu lelah habis jalan jauh tunggu luar karena slotcard pintu kamar di perbaikin ulang tks smoga manfaat
2. tak sesuai espektasi kamar sempit pintu kamar mandi rusak cek in tunggu terlalu lama malam suara berisik dr atap kamar nyaman
3. sangat emejing inap airy sutoyo hotel dewarnaudah murah tempat nyaman buat istirahatapalagi breakfastnya nikmat ga asal harga murah kualitas murahantp yg perlu baik cuman bersih kamar kurang bersih klo ngepeloverall rekomen bgt
4. ac nya parah berisik bersurara bikin gak tidur sama sekali lokasi nya susah minta ampun temu
5. kamar bau selimut handuk kusam staffnya gak ramah kamar tdk sesuai ekspektasi sempit pengap breakfast kayak makan kucing jd cari sarap luar harga segitu gak worth it


### Evaluating

for the purpose of this project, we will be using ROUGE and BLEU metrics

#### BLEU Metrics

There are libraries for BLEU metrics in nltk. Thus, in this project, we will use them.

Difference between corpus_bleu and sentence_bleu:
    1. corpus_bleu: to evaluate some sentences in a paragraph or document
    2. sentence_bleu: to evaluate one sentence against some reference sentences

In [64]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu

references = ["tak sesuai ekspektasi kamar pengap"]

idx = 0
print("BLEU Metrics Summary Evaluation")
print()

for candidate in candidates:
    candidate.split()
    print(candidate)
    score = sentence_bleu(references, candidate)
    print("%s: %s" % ("score for this summary: ", score))
    print()

BLEU Metrics Summary Evaluation

kamar bersih nyaman perlu perhati dan trus kroscheck trutama slot card pintu kamar batrai nya habis apa msih full kasi ada tamu lelah habis jalan jauh tunggu luar karena slotcard pintu kamar di perbaikin ulang tks smoga manfaat
score for this summary: : 0.06138813788678699

tak sesuai espektasi kamar sempit pintu kamar mandi rusak cek in tunggu terlalu lama malam suara berisik dr atap kamar nyaman
score for this summary: : 0.21271927853112058

sangat emejing inap airy sutoyo hotel dewarnaudah murah tempat nyaman buat istirahatapalagi breakfastnya nikmat ga asal harga murah kualitas murahantp yg perlu baik cuman bersih kamar kurang bersih klo ngepeloverall rekomen bgt
score for this summary: : 0.057512116804495854

ac nya parah berisik bersurara bikin gak tidur sama sekali lokasi nya susah minta ampun temu
score for this summary: : 0.08157811732948411

kamar bau selimut handuk kusam staffnya gak ramah kamar tdk sesuai ekspektasi sempit pengap breakfast k

#### ROUGE Metrics

There are 3 types of ROUGE Metrics:
1. ROUGE-N – measures unigram, bigram, trigram and higher order n-gram overlap. ROUGE-N may consist of ROUGE-1 and ROUGE-2, where it analyzes unigram and bigram overlap respectively.
2. ROUGE-L –  measures longest matching sequence of words using LCS. An advantage of using LCS is that it does not require consecutive matches but in-sequence matches that reflect sentence level word order. Since it automatically includes longest in-sequence common n-grams, you don’t need a predefined n-gram length.
3. ROUGE-S – Is any pair of word in a sentence in order, allowing for arbitrary gaps. This can also be called skip-gram coocurrence. For example, skip-bigram measures the overlap of word pairs that can have a maximum of two gaps in between words. As an example, for the phrase “cat in the hat” the skip-bigrams would be “cat in, cat the, cat hat, in the, in hat, the hat”. 


Quoted from with some changes:
https://kavita-ganesan.com/what-is-rouge-and-how-it-works-for-evaluation-of-summaries/#.XpQIZ2wzZuR

##### Consideration for choosing ROUGE-1:
We are developing summary with extractive summarization method with fairly verbose system and reference summaries, then it may make sense to use ROUGE-1 and ROUGE-L. For very concise summaries, ROUGE-1 alone may suffice especially if you are also applying stemming and stop word removal.


In [67]:
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg', 'Best', 'Individual']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

    references_1 = "tak sesuai ekspektasi kamar pengap"
    references_2 = "ac nya parah berisik bersurara bikin gak tidur"
    references_3 = "slotcard pintu kamar di perbaikin ulang"
    
    all_hypothesis = [candidate[0], candidate[1], candidate[2], candidate[3], candidate[4]]
    all_references = [references_1, references_2]

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg


TypeError: __init__() got an unexpected keyword argument 'max_n'

## Main Reference

- Preprocessing techniques ideas are inspired by the links below: 
    1. https://www.academia.edu/35015140/Preprocessing_Techniques_for_Text_Mining
    2. https://medium.com/@ksnugroho/dasar-text-preprocessing-dengan-python-a4fa52608ffe

- Reference for data shortening:
    1. https://stackoverflow.com/questions/10249673/python-read-lines-of-website-source-code-100-lines-at-a-time
    
- References for library for Indonesian Stopwords: 
    1. https://medium.com/@ksnugroho/dasar-text-preprocessing-dengan-python-a4fa52608ffe
    2. https://devtrik.com/python/stopword-removal-bahasa-indonesia-python-sastrawi/
    
- Code for the modelling of the project is mainly referred from this website: 
    1. https://nlpforhackers.io/textrank-text-summarization/ 
    2. I do not take ownership of the code presented above
    
- ROUGE Metrics:
    1. https://pypi.org/project/py-rouge/

- BLEU Metrics:
    1. https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

## Additional Reference (for reading references only)

Below are links I have been referring to throughout the making of the project and want to revisit in the future:
- TextRank algorithms: 
    1. https://iq.opengenus.org/textrank-for-text-summarization/
    2. https://github.com/summanlp/textrank
    3. https://github.com/davidadamojr/TextRank
    4. https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf
    5. https://medium.com/@shivangisareen/text-summarisation-with-gensim-textrank-46bbb3401289
    6. https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70
- Text Summarization algorithm using word frequency: 
    1. https://becominghuman.ai/text-summarization-in-5-steps-using-nltk-65b21e352b65
- Metrics (BLEU):
    1. https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
    2. http://www.nltk.org/_modules/nltk/translate/bleu_score.html
    3. https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213
    4. http://www.nltk.org/api/nltk.translate.html
    5. https://www.aclweb.org/anthology/P02-1040.pdf
- Metrics (ROUGE):
    1. https://github.com/pltrdy/rouge
    2. https://kavita-ganesan.com/what-is-rouge-and-how-it-works-for-evaluation-of-summaries/#.XpQIZ2wzZuR
    3. https://www.aclweb.org/anthology/W04-1013.pdf